In [4]:
library(taxizedb)
library(data.table)
library(dplyr)
library(tidyr)

In [32]:
#hgtector assignments
hg_ass <- fread("/mnt/workspace/analyses/omics/hgtector/analysis/resources/class_based_on_diamond.txt", header = F, sep = ":") %>%
    mutate(V2 = sub(pattern = "All input genomes belong to", replacement = "", V2),
          V1 = sub(pattern = "wd_0412/analyses/", replacement = "", V1),
          V1 = sub(pattern = "_info.txt", replacement = "", fixed = T, V1),
          ) %>%
    rowwise() %>%
    mutate(V2 = strsplit(x = V2, split = "(", fixed = T)[[1]][[1]]) %>%
    select(bin=V1, taxid = V2) %>%
    mutate(taxid = as.integer(taxid))

    
hg_ass


bin,taxid
<chr>,<int>
nod_bin.10,2800788
nod_bin.11,31989
nod_bin.12,356
nod_bin.13,1649453
nod_bin.14,2933270
nod_bin.15,68336
nod_bin.16,80840
nod_bin.17,85023
nod_bin.18,1633335


In [45]:
get_lineage2 <- function(x){
    standard_ranks <- c("kingdom", "phylum", "class", "order", "family", "genus", "species")
    tax_info <- classification(x, db="ncbi")
    # Print the fetched data
    #tax_info
    tt <- lapply(x, function(x){
        tax_info <- classification(x, db="ncbi")  
        df <- tax_info[[1]]
        df$query <- x
        return(df)
    })
    dfo <- bind_rows(tt) %>%
        mutate(name=paste(name,id," ")) %>%
        select(-id) %>%
    pivot_wider(names_from = rank, values_from = name)
    return(dfo)
}
tt <- get_lineage2(hg_ass$taxid)
df_transformed <- tt %>%
  mutate(across(-1, ~sapply(., function(x) paste(x, collapse = ", "), simplify = "character")))

hg_ass2 <- hg_ass %>%
    left_join(df_transformed, by = c("taxid"="query")) %>%
    rowwise() %>%
    mutate(binnr = as.integer(strsplit(split = ".", fixed = T, x = bin)[[1]][[2]])) %>%
    arrange(binnr) %>%
    select(-binnr)
                                
hg_ass2

Warning message:
“Values from `name` are not uniquely identified; output will contain list-cols.
• Use `values_fn = list` to suppress this warning.
• Use `values_fn = {summary_fun}` to summarise duplicates.
• Use the following dplyr code to identify duplicates.
  {data} |>
  dplyr::summarise(n = dplyr::n(), .by = c(query, rank)) |>
  dplyr::filter(n > 1L)”


bin,taxid,no rank,superkingdom,phylum,class,order,family,clade,genus,species,strain
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
nod_bin.1,690086,cellular organisms 131567,Bacteria 2,Pseudomonadota 1224,Alphaproteobacteria 28211,Hyphomicrobiales 356,Salinarimonadaceae 2137278,,Salinarimonas 690086,,
nod_bin.2,1925762,cellular organisms 131567,Bacteria 2,Bacteroidota 976,Cytophagia 768503,Cytophagales 768507,Cyclobacteriaceae 563798,"FCB group 1783270 , Bacteroidota/Chlorobiota group 68336",Algoriphagus 246875,Algoriphagus marinus 1925762,
nod_bin.3,1853230,cellular organisms 131567,Bacteria 2,Bacteroidota 976,Flavobacteriia 117743,Flavobacteriales 200644,Crocinitomicaceae 1853230,"FCB group 1783270 , Bacteroidota/Chlorobiota group 68336",,,
nod_bin.4,2609175,cellular organisms 131567,Bacteria 2,Pseudomonadota 1224,Alphaproteobacteria 28211,Maricaulales 2800059,Maricaulaceae 2800061,,Alkalicaulis 2800082,Alkalicaulis satelles 2609175,
nod_bin.5,28211,cellular organisms 131567,Bacteria 2,Pseudomonadota 1224,Alphaproteobacteria 28211,,,,,,
nod_bin.6,1914872,cellular organisms 131567,Bacteria 2,Cyanobacteriota 1117,Cyanophyceae 3028117,Nostocales 1161,Aphanizomenonaceae 1892259,"Terrabacteria group 1783272 , Cyanobacteriota/Melainabacteria group 1798711",Nodularia 159191,Nodularia spumigena 70799,Nodularia spumigena UHCC 0039 1914872
nod_bin.7,2,cellular organisms 131567,Bacteria 2,,,,,,,,
nod_bin.8,203683,cellular organisms 131567,Bacteria 2,Planctomycetota 203682,Planctomycetia 203683,,,PVC group 1783257,,,
nod_bin.9,411684,cellular organisms 131567,Bacteria 2,Pseudomonadota 1224,Alphaproteobacteria 28211,Hyphomicrobiales 356,Phyllobacteriaceae 69277,,Hoeflea 274591,Hoeflea phototrophica 244596,Hoeflea phototrophica DFL-43 411684


In [125]:
dfg <- fread("/mnt/workspace/analyses/omics/hgtector/analysis/wd_0412/analyses/most_abundant_groups_assignments.tsv")
names(dfg)
dfg %>%
    filter(bin == "bin.18" &
          grepl("Thalassobaculaceae", family)     
          ) %>%
        # select(family,N) %>%
        # group_by(family) %>%
        # summarise(N=sum(N)) %>%
        # arrange(-N) %>%
        
    head()

[1] "phylum" "class"  "order"  "family" "genus"  "N"      "bin"

phylum,class,order,family,genus,N,bin
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>
Pseudomonadota 1224,Alphaproteobacteria 28211,Rhodospirillales 204441,Thalassobaculaceae 2844864,Nisaea 390876,4,bin.18
Pseudomonadota 1224,Alphaproteobacteria 28211,Rhodospirillales 204441,Thalassobaculaceae 2844864,Thalassobaculum 526215,4,bin.18


In [47]:
g_summary <- fread("genome.summary") %>%
    rowwise() %>%
    mutate(Species=strsplit(`GTDB-Taxa`,split=";")[[1]][[7]]) %>%
    mutate(Genus=strsplit(`GTDB-Taxa`,split=";")[[1]][[6]]) %>%
    mutate(Family=strsplit(`GTDB-Taxa`,split=";")[[1]][[5]]) %>%
    mutate(Order=strsplit(`GTDB-Taxa`,split=";")[[1]][[4]]) %>%
    select(-`GTDB-Taxa`) %>%
    mutate(
        Species = sub("s__","",Species),
        Genus = sub("g__","",Genus),
        Family = sub("f__","",Family),
        Order = sub("o__","",Order)
    )
#
#ajust manualy genus
g_summary$Genus[g_summary$Genus=="CAU-1598"]="Pseudomarimonas"
g_summary$Order[g_summary$Order=="Rhizobiales"]="Hyphomicrobiales"

g_summary
#Hyphomicrobiales - Rhisobiales now

#BinID,Completeness,Contamination,Strain heterogeneity,GenomeSize(bp),N_Contig,N50(bp),GC,Species,Genus,Family,Order
<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>
bin.1,98.43,0.99,0.00,5281460,1,5281460,0.71459,,Salinarimonas,Beijerinckiaceae,Hyphomicrobiales
bin.2,100.00,0.00,0.00,3962322,1,3962322,0.39833,,Algoriphagus,Cyclobacteriaceae,Cytophagales
bin.3,86.56,2.70,80.00,3520134,74,67937,0.36345,,,Crocinitomicaceae,Flavobacteriales
bin.4,99.45,1.14,0.00,3233947,1,3233947,0.66182,,Oceanicaulis,Maricaulaceae,Caulobacterales
bin.5,97.19,0.20,0.00,2908195,2,2362761,0.65060,,MH13,MH13,Hyphomicrobiales
bin.6,98.78,0.00,0.00,5752838,5,5461907,0.41037,Nodularia,Nodularia,Nostocaceae,Cyanobacteriales
bin.7,86.93,0.00,0.00,3246209,1,3246209,0.64849,,UBA6054,UBA1924,Phycisphaerales
bin.8,85.46,1.44,33.33,6176557,41,322590,0.59406,,M30B19,Pirellulaceae,Pirellulales
bin.9,97.47,0.30,25.00,4104996,6,620707,0.59683,,Hoeflea,Rhizobiaceae,Hyphomicrobiales


In [44]:
get_lineages <- function(x){
    standard_ranks <- c("kingdom", "phylum", "class", "order", "family", "genus", "species")
    tax_info <- classification(ids, db="ncbi")
    # Print the fetched data
    #tax_info
    tt <- lapply(x, function(x){
        tax_info <- classification(x, db="ncbi")  
        df <- tax_info[[1]]
        df$query <- x
        return(df)
    })
    dfo <- bind_rows(tt) %>%
        filter(rank %in% standard_ranks) %>%
        select(-id) %>%
    pivot_wider(names_from = rank, values_from = name)
    return(dfo)
}


In [45]:
genus <- lapply(X = g_summary$Genus,FUN=function(x){
    q <- name2taxid(x=x,out_type = "summary")
    return(q)
})

genus <- bind_rows(genus) %>%
    distinct()
genus_lineages <- get_lineages(genus$id) %>%
    rename(id=query) %>%
    filter(is.na(kingdom)) %>%
    select(genus_id = id, genus)

g_summary2 <-g_summary %>%
    left_join(genus_lineages, by = c("Genus"="genus"))
#ajust
g_summary2

#BinID,Completeness,Contamination,Strain heterogeneity,GenomeSize(bp),N_Contig,N50(bp),GC,Species,Genus,Family,Order,genus_id
<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
bin.1,98.43,0.99,0.00,5281460,1,5281460,0.71459,,Salinarimonas,Beijerinckiaceae,Rhizobiales,690086
bin.2,100.00,0.00,0.00,3962322,1,3962322,0.39833,,Algoriphagus,Cyclobacteriaceae,Cytophagales,246875
bin.3,86.56,2.70,80.00,3520134,74,67937,0.36345,,,Crocinitomicaceae,Flavobacteriales,NA
bin.4,99.45,1.14,0.00,3233947,1,3233947,0.66182,,Oceanicaulis,Maricaulaceae,Caulobacterales,153232
bin.5,97.19,0.20,0.00,2908195,2,2362761,0.65060,,MH13,MH13,Rhizobiales,NA
bin.6,98.78,0.00,0.00,5752838,5,5461907,0.41037,Nodularia,Nodularia,Nostocaceae,Cyanobacteriales,159191
bin.7,86.93,0.00,0.00,3246209,1,3246209,0.64849,,UBA6054,UBA1924,Phycisphaerales,NA
bin.8,85.46,1.44,33.33,6176557,41,322590,0.59406,,M30B19,Pirellulaceae,Pirellulales,NA
bin.9,97.47,0.30,25.00,4104996,6,620707,0.59683,,Hoeflea,Rhizobiaceae,Rhizobiales,274591


In [5]:
get_close <- function(x) {
    id = names(tax_ids)[grepl(pattern = x, names(tax_ids))]
    print(x)
    if (length(id) == 1) {
        return(tax_ids[[id]])
    } else {tax_ids[[id[[1]]]]}
    #return(tax_ids[[id]])
    

}

g_summary2 <- g_summary %>%
    rowwise() %>%
    mutate(Family_txid = if_else(is.na(Family_txid),get_close(Family),Family_txid)) %>%
    mutate(Genus_txid = if_else(is.na(Genus_txid),get_close(Genus),Genus_txid))
    #mutate(Family_txid = if_else(is.na(Family_txid),get_close(Family_txid),Family_txid))
g_summary2

ERROR: [1m[33mError[39m in `mutate()`:[22m
[1m[22m[36mℹ[39m In argument: `Family_txid = if_else(is.na(Family_txid), get_close(Family), Family_txid)`.
[36mℹ[39m In row 1.
[1mCaused by error:[22m
[33m![39m object 'Family_txid' not found


In [67]:
get_close <- function(x) {
    id = names(tax_ids)[grepl(pattern = x, names(tax_ids))][[1]]
    return(tax_ids[[id]])
}
get_close("HLUCCA01")

[1] 1666909

In [94]:
# get genome summary update
local_file_path <- "./resources/assembly_summary_refseq.txt"

assembly_data <- fread(local_file_path, skip = 1, sep = "\t", header = TRUE, quote = "")

In [108]:
tt <- assembly_data %>%
    group_by(taxid) %>%
    summarise(N=n()) %>%
    arrange(-N)
filter(assembly_data, species_taxid == 158836)

#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,⋯,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id
<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,⋯,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GCF_900075105.1,PRJNA224116,SAMEA2273507,FJWM00000000.1,na,158836,158836,Enterobacter hormaechei,strain=e1227,na,⋯,0,59,59,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023/03/25,4797,4614,87,na
GCF_900075765.1,PRJNA224116,SAMEA2273362,FJZC00000000.1,na,158836,158836,Enterobacter hormaechei,strain=e1486,na,⋯,0,46,46,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023/03/25,4884,4723,91,na
GCF_900075195.1,PRJNA224116,SAMEA2273307,FJXS00000000.1,na,158836,158836,Enterobacter hormaechei,strain=e1213,na,⋯,0,115,115,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023/03/25,4789,4571,84,na
GCF_900075845.1,PRJNA224116,SAMEA2273376,FJZK00000000.1,na,158836,158836,Enterobacter hormaechei,strain=e1547,na,⋯,0,23,23,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023/03/25,4563,4437,90,na
GCF_900075635.1,PRJNA224116,SAMEA2273345,FJYQ00000000.1,na,158836,158836,Enterobacter hormaechei,strain=e1386,na,⋯,0,71,71,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023/03/25,4850,4658,86,na
GCF_900075835.1,PRJNA224116,SAMEA2273373,FJZJ00000000.1,na,158836,158836,Enterobacter hormaechei,strain=e1531,na,⋯,0,51,51,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023/03/25,4814,4647,85,na
GCF_900075355.1,PRJNA224116,SAMEA2273286,FJWV00000000.1,na,158836,158836,Enterobacter hormaechei,strain=e1069,na,⋯,0,43,43,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023/03/25,4654,4491,92,na
GCF_900075305.1,PRJNA224116,SAMEA2273213,FJWR00000000.1,na,158836,158836,Enterobacter hormaechei,strain=e1105,na,⋯,0,78,78,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023/03/25,5131,4942,84,na
GCF_900075665.1,PRJNA224116,SAMEA2273347,FJYT00000000.1,na,158836,158836,Enterobacter hormaechei,strain=e1391,na,⋯,0,43,43,NCBI RefSeq,NCBI Prokaryotic Genome Annotation Pipeline (PGAP),2023/03/25,4778,4610,97,na
